In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [3]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.05
# config.gpu_options.visible_device_list = "0" #only the gpu 0 is allowed
set_session(tf.Session(config=config))

In [4]:
import random
from os import listdir
from os.path import isfile, join

import numpy as np

import json
import matplotlib.pyplot as plt
import cv2
margin = 0.2
from tqdm import tqdm
from mtcnn import MTCNN
import pickle

In [5]:
detector = MTCNN()


# Extract face from kaggle data set

In [6]:
# path0 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_0"
# path1 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_1"
# path2 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_2"
# path3 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_3"
# path48 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_48"
# path49 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_49"

# path4 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_4"
# path5 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_5"
# path6 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_6"
# path7 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_7"
# path8 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_8"
# path9 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_9"


# path16 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_16"
# path17 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_17"
# path18 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_18"
# path19 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_19"
# path20 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_20"

path36 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_36"
path37 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_37"
path38 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_38"
path39 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_39"
path40 = "/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_40"

In [7]:
paths = [path36,path37,path38,path39,path40]

In [8]:
# data = json.load(open("/hdd/tam/kaggle/train_sample_videos/metadata.json"))
# data0 = json.load(open("/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_48/metadata.json"))
# data1 = json.load(open("/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_48/metadata.json"))
# data2 = json.load(open("/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_48/metadata.json"))
# data3 = json.load(open("/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_48/metadata.json"))
# data48 = json.load(open("/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_48/metadata.json"))
# data49 = json.load(open("/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_48/metadata.json"))
IMGWIDTH = 128
margin = 0.2

In [9]:
# df =0
# real = 0
# for vi in tqdm(data):
#     if data[vi]['label'] == "FAKE":
#         df+=1
#     if data[vi]['label'] == 'REAL':
#         real+=1

In [10]:
os.path.exists(join("/hdd/tam/kaggle/train_videos/real", "ncmpqwmnzb.mp4"+".pkl"))

True

In [ ]:
# save_interval = 6 # perform face detection every {save_interval} frames
for path in paths:
    data = json.load(open(join(path, "metadata.json")))
    print(path)
    for vi in tqdm(data): 
#         if data[vi]['label'] == "REAL":
#             continue
        if data[vi]['label'] == "FAKE":
            if os.path.exists(join("/hdd/tam/kaggle/train_videos/df", vi +".pkl")):
                continue
        if data[vi]['label'] == 'REAL':
            if os.path.exists(join("/hdd/tam/kaggle/train_videos/real", vi+".pkl")):
                continue
            
            
        video = cv2.VideoCapture(join(path, vi))
        success = True
    #     success, vframe = video.read()
        data_videos = []
        save_interval = 13

        success, image = video.read()
        while success:
    #         for i in range(0,video.__len__(),save_interval):
            for i in range(save_interval):
                success, image = video.read()
                if not success:
                    break
    #         print(image.shape)
    #         if image.all() ==None:
    #             continue
            try:
                image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            except:
                continue
        #         face_positions = face_recognition.face_locations(img)
            face_positions = detector.detect_faces(image)
            if len(face_positions) == 0:
                continue
            face_position =face_positions[0]['box']
            x,y,w,h = face_position
            offsetx = round(margin * (w))
            offsety = round(margin * (h))
            y0 = round(max(y - offsety, 0))
            x1 = round(min(x + w + offsetx, image.shape[1]))
            y1 = round(min(y+ h + offsety, image.shape[0]))
            x0 = round(max(x - offsetx, 0))
    #         print(x0,x1,y0,y1)
            face = image[y0:y1,x0:x1]


            face = cv2.resize(face,(IMGWIDTH,IMGWIDTH))
    #         plt.imshow(face)
    #         plt.show()
            data_videos.append(face)
            success, image = video.read()

        data_videos = np.array(data_videos)
        if data[vi]['label'] == "FAKE":
            output = open(join("/hdd/tam/kaggle/train_videos/df", vi +".pkl"),'wb')
            pickle.dump(data_videos, output)
            output.close()
        if data[vi]['label'] == 'REAL':
            output = open(join("/hdd/tam/kaggle/train_videos/real", vi+".pkl"),'wb')
            pickle.dump(data_videos, output)
            output.close()

  0%|          | 0/2655 [00:00<?, ?it/s]

/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_36
/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_37


  0%|          | 0/2477 [00:00<?, ?it/s]

/hdd/tam/kaggle/www.kaggle.com/c/16880/datadownload/dfdc_train_part_38


  7%|▋         | 171/2477 [31:17<6:27:20, 10.08s/it]

# Extract face from FaceForensics

In [12]:
import glob

In [13]:
data = glob.glob("/hdd/tam/FaceForensics/data/original_sequences/youtube/c23/videos/*.mp4")

In [14]:
len(data)

1000

In [ ]:
for vi in tqdm(data):
    vi = vi.split("/")[-1]
    video = cv2.VideoCapture(join("/hdd/tam/FaceForensics/data/original_sequences/youtube/c23/videos/", vi))
    success = True
#     success, vframe = video.read()
    data_videos = []
    save_interval = 7
    
    success, image = video.read()
    while success:
        for i in range(save_interval):
            success, image = video.read()
            if not success:
                break
        try:
            image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        except:
            continue
    #         face_positions = face_recognition.face_locations(img)
        face_positions = detector.detect_faces(image)
        if len(face_positions) == 0:
            continue
        face_position =face_positions[0]['box']
        x,y,w,h = face_position
        offsetx = round(margin * (w))
        offsety = round(margin * (h))
        y0 = round(max(y - offsety, 0))
        x1 = round(min(x + w + offsetx, image.shape[1]))
        y1 = round(min(y+ h + offsety, image.shape[0]))
        x0 = round(max(x - offsetx, 0))
#         print(x0,x1,y0,y1)
        face = image[y0:y1,x0:x1]


        face = cv2.resize(face,(IMGWIDTH,IMGWIDTH))
#         plt.imshow(face)
#         plt.show()
        data_videos.append(face)
        success, image = video.read()

    data_videos = np.array(data_videos)
#     print(data_videos.shape)
    output = open(join("/hdd/tam/kaggle/train_videos/real", vi+".pkl"),'wb')
    pickle.dump(data_videos, output)
    output.close()

  2%|▏         | 18/1000 [04:25<3:35:21, 13.16s/it]

In [ ]:
1

In [16]:
from ipywidgets import Video

In [18]:
video = Video.from_file("/hdd/tam/FaceForensics/data/original_sequences/youtube/c23/videos/668.mp4")
video

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x0e\x8f\xc4mdat\x0…